# Importing and Preparing the dataset

In [1]:
import pandas as pd

# df = pd.read_pickle('T_combined_final_double_selector.pkl')
df = pd.read_pickle('classification_dataset.pkl')
df.dropna(inplace=True)
df.reset_index(drop=True, inplace = True)
df


,fps_blend_comp,bdid,c1,prop,val_1,val_2
0,"[1.2037544532748699, 1.852957906712173, 1.1290...",547,0.8000,Tg,238.15,238.15
1,"[1.2640287769784173, 1.6689081011847582, 1.076...",547,0.5000,Tg,248.15,248.15
2,"[1.223196881091618, 1.7872573897537893, 1.1111...",547,0.7000,Tg,249.15,249.15
3,"[1.2640287769784173, 1.6689081011847582, 1.076...",547,0.5000,Tg,338.15,338.15
4,"[1.2037544532748699, 1.852957906712173, 1.1290...",547,0.8000,Tg,342.15,342.15
...,...,...,...,...,...,...
4505,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.032...",2045,0.7500,Td,643.15,643.15
4506,"[1.2449499208854404, 1.5672387794174951, 1.0, ...",2051,0.4970,Td,629.15,629.15
4507,"[1.244969919920322, 1.567610622468521, 1.0, 1....",2051,0.4990,Td,630.65,630.65
4508,"[1.244959920322565, 1.5674246788897548, 1.0, 1...",2051,0.4980,Td,631.15,631.15


In [2]:
import numpy as np
df['fps_blend_comp'] = (np.vstack(df.fps_blend_comp) - 1).tolist()

In [3]:
df['fps_blend_comp'] = (np.around(np.vstack(df.fps_blend_comp),decimals=5 )).tolist()

In [4]:
df['miscibility'] = (df.val_1 == df.val_2)*1 # Singled value = 1 'miscible'

In [5]:
df = df[df['prop'] != 'Td']
df.insert(0, 'selector', pd.get_dummies(df.prop).values.tolist())


In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [9]:
df.reset_index(drop = True, inplace=True)

In [ ]:
df.fillna(0, inplace=True)

In [16]:
df

,selector,fps_blend_comp,bdid,c1,prop,val_1,val_2,miscibility
0,"[1, 0]","[0.20375, 0.85296, 0.12903, 0.27517, 0.25, 0.4...",547,0.80,Tg,238.15,238.15,1
1,"[1, 0]","[0.26403, 0.66891, 0.07692, 0.19685, 0.14286, ...",547,0.50,Tg,248.15,248.15,1
2,"[1, 0]","[0.2232, 0.78726, 0.11111, 0.24795, 0.21212, 0...",547,0.70,Tg,249.15,249.15,1
3,"[1, 0]","[0.26403, 0.66891, 0.07692, 0.19685, 0.14286, ...",547,0.50,Tg,338.15,338.15,1
4,"[1, 0]","[0.20375, 0.85296, 0.12903, 0.27517, 0.25, 0.4...",547,0.80,Tg,342.15,342.15,1
...,...,...,...,...,...,...,...,...
4244,"[0, 1]","[0.24404, 0.5775, 0.0, 0.07748, 0.0, 0.7198, 0...",2151,0.28,Tm,421.75,421.75,1
4245,"[0, 1]","[0.18245, 0.58736, 0.0, 0.23073, 0.0, 0.5371, ...",2151,0.73,Tm,422.65,422.65,1
4246,"[0, 1]","[0.20499, 0.58362, 0.0, 0.16797, 0.0, 0.60137,...",2151,0.56,Tm,422.75,422.75,1
4247,"[0, 1]","[0.21712, 0.60635, 0.0, 0.14943, 0.00689, 0.63...",2152,0.52,Tm,424.25,424.25,1


# Converting the dataset for training the model using tensorflow

In [ ]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import numpy as np
from sklearn.preprocessing import QuantileTransformer, MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
import pandas as pd
import tensorflow as tf
tf.random.set_seed(123)


df = df.sample(frac=1, random_state=123)

target_property_scaler = MinMaxScaler

batch_size = 200

skf = StratifiedKFold(n_splits=5)

props = df.prop.unique().tolist()

datasets = []
def get_dataset(df):
    fps_blends = np.stack(df.fps_blend_comp.values).astype(np.float32)

    selector = np.stack(df.selector.values).astype(np.float32)
    target = df.miscibility.values.astype(int)[:, np.newaxis]
    bdid = np.stack(df.bdid.values).astype(np.str)[:, np.newaxis]
    # miscibility = df.miscibility.values.astype(int)[:, np.newaxis]
    c1 = df.c1.values.astype(np.float32)[:, np.newaxis]
    # type_poly = np.stack(df.type.values).astype(np.str)[:, np.newaxis]
    dataset = tf.data.Dataset.from_tensor_slices(({'selector': selector, 'fps_blends': fps_blends, 'prop': df.prop, 'c1': c1, 'bdid':bdid}, target))

    dataset = dataset.cache().batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    return dataset


training_df, test_df = train_test_split(df, test_size=0.2, stratify=df.prop, random_state=123)
training_df, test_df = training_df.copy(), test_df.copy()

for train_index, val_index in skf.split(training_df, training_df.prop):
    
    train_df = df.iloc[train_index].copy()
    val_df = df.iloc[val_index].copy()
    
    
    datasets.append({'train': get_dataset(train_df), 'val': get_dataset(val_df)})
datasets_final = {'train': get_dataset(training_df), 'test': get_dataset(test_df)}



In [19]:
datasets_final

{'train': <PrefetchDataset element_spec=({'selector': TensorSpec(shape=(None, 2), dtype=tf.float32, name=None), 'fps_blends': TensorSpec(shape=(None, 872), dtype=tf.float32, name=None), 'prop': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'c1': TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), 'bdid': TensorSpec(shape=(None, 1), dtype=tf.string, name=None)}, TensorSpec(shape=(None, 1), dtype=tf.int64, name=None))>,
 'test': <PrefetchDataset element_spec=({'selector': TensorSpec(shape=(None, 2), dtype=tf.float32, name=None), 'fps_blends': TensorSpec(shape=(None, 872), dtype=tf.float32, name=None), 'prop': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'c1': TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), 'bdid': TensorSpec(shape=(None, 1), dtype=tf.string, name=None)}, TensorSpec(shape=(None, 1), dtype=tf.int64, name=None))>}

# Define model

In [26]:
import tensorflow.keras as tfk
import tensorflow as tf
from datetime import datetime 
from tensorflow.python.keras.engine import data_adapter
from keras_tuner import HyperModel
from keras_tuner.tuners import Hyperband, RandomSearch
import tensorflow_addons as tfa
from tensorflow import keras

df.fps_blend_comp.to_pickle('pkl/fingerprints_combined_classification_final.pkl')
class MetaModel(tfk.Model):
    def __init__(self, hp): #, scale):
        super().__init__()
        self.base_models = []
        # self.scaler_type = scale
        for num in range(5):
            model = tf.keras.models.load_model(f'models/cv_final_class/{num}')
            model.trainable = False
            self.base_models.append(model)
        
        self.my_layers = []
        
        for i in range(hp.Int('num_layers', 1, 2)): 
            new_step = [               
            tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=64,
                                            max_value=544,
                                            step=64),),
            
            tf.keras.layers.PReLU(),
            tf.keras.layers.Dropout(hp.Float(
                'dropout_' + str(i),
                min_value=0.0,
                max_value=0.5,
                default=0.25,
                step=0.05,
            )),]
            self.my_layers.append(new_step)
            
        self.my_layers.append([tf.keras.layers.Dense(1, activation = 'sigmoid')])

    def call(self, inputs, training=None): 
        
        # drop prop
        if 'prop' in inputs:
            del inputs['prop']
        x = []
        for base in self.base_models:
            if training:
                res = base.call(inputs, training)
            else:
                res = base.call(inputs)
            x.append(res)
        x = tf.concat(x, -1)
        
        for num, layer_step in enumerate(self.my_layers):
            for layer in layer_step:
                x = layer(x)

        return x
    
    def predict_step(self, data):
        data = data_adapter.expand_1d(data)
        x, _, _ = data_adapter.unpack_x_y_sample_weight(data)
        # drop prop here
        prop = x['prop']
        # bdid = x['bdid']
        # type_poly = x['type']
        selector = x['selector']
        bdid = x['bdid']
        c1 = x['c1']
        # miscibility = x['miscibility']
        del x['prop']
        # del x['bdid']
        # del x['selector']
        return self(x, training=True), data[-1], prop, bdid, c1, selector

    
    @tf.function(input_signature=[])
    def fp_order(self):
        fp_order = tf.convert_to_tensor(list(pd.read_pickle('pkl/fingerprints_combined_classification_final.pkl').columns))
        return fp_order
        
    
def build_model(hp):
    model = MetaModel(hp)
    opt = tf.keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-3]))
    opt = tfa.optimizers.SWA(opt)

    model.compile(
        optimizer=opt,
        loss=keras.losses.BinaryCrossentropy(from_logits=False),)
    return model

# Hyperparameter optimization

In [ ]:
import IPython
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt


results, property_metric = [], []
prec_Tg = []
recall_Tg = []
accuracy_Tg = []
prec_Tm = []
recall_Tm = []
accuracy_Tm = []


tuner = Hyperband(
    build_model,
    objective='val_loss',
    max_epochs=300,
    # hyperband_iterations=3,
    seed=10,
    directory=f'hyperparameter_search_meta_learner_mt_class_final',
    project_name='fold_0',
    )

reduce_lr = tfk.callbacks.ReduceLROnPlateau(
    factor=0.9,
    monitor="val_loss",
    verbose=1,
)

earlystop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=40)

class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait = True)

# Create an instance of the model
tuner.search(datasets_final['test'],
            epochs=300,
            validation_data=datasets_final['train'],
            callbacks=[earlystop, reduce_lr, ClearTrainingOutput()],
            verbose=0
            )

# Post processing
best_values = tuner.get_best_hyperparameters()[0].values

best_model = tuner.get_best_models(1)[0]
res = np.concatenate(best_model.predict(datasets_final['train']), -1)
# best_model.save(f'models/meta_model_{what}/', include_optimizer=False)


_df = pd.DataFrame(res, columns=['pred', 'target', 'prop', 'bdid','c1', 'selector_0', 'selector_1'])
_df['prop'] = _df.prop.apply(lambda x: x.decode('utf-8'))
# _df['bdid'] = _df.prop.apply(lambda x: x.decode('utf-8'))

props = _df.prop.unique()
res_Tg = _df[_df['prop'] == 'Tg']
res_Tm = _df[_df['prop'] == 'Tm']      

y_true = np.array(res_Tm.target.values, dtype = int)
y_scores = res_Tm.pred.values
fpr, tpr, thresholds = metrics.roc_curve(y_true, y_scores, pos_label = 1)
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]
predicted_value = []
for i in res_Tm.pred.values:
    if i > optimal_threshold:
        predicted_value.append(1)
    else:
        predicted_value.append(0)
m = tf.keras.metrics.Precision()
m.update_state(np.array(res_Tm.target.values, dtype = int), predicted_value)
prec_Tm.append(m.result().numpy())
m = tf.keras.metrics.Recall()
m.update_state(np.array(res_Tm.target.values, dtype = int), predicted_value)
recall_Tm.append(m.result().numpy())
m = tf.keras.metrics.Accuracy()
m.update_state(np.array(res_Tm.target.values, dtype = int), predicted_value)
accuracy_Tm.append(m.result().numpy())

y_true = np.array(res_Tg.target.values, dtype = int)
y_scores = res_Tg.pred.values
fpr, tpr, thresholds = metrics.roc_curve(y_true, y_scores, pos_label = 1)
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]
predicted_value = []
for i in res_Tg.pred.values:
    if i > optimal_threshold:
        predicted_value.append(1)
    else:
        predicted_value.append(0)
m = tf.keras.metrics.Precision()
m.update_state(np.array(res_Tg.target.values, dtype = int), predicted_value)
prec_Tg.append(m.result().numpy())
m = tf.keras.metrics.Recall()
m.update_state(np.array(res_Tg.target.values, dtype = int), predicted_value)
recall_Tg.append(m.result().numpy())
m = tf.keras.metrics.Accuracy()
m.update_state(np.array(res_Tg.target.values, dtype = int), predicted_value)
accuracy_Tg.append(m.result().numpy())


In [30]:
df_performance = pd.DataFrame({'prec_Tg' :prec_Tg, 'recall_Tg' : recall_Tg, 'accuracy_Tg' : accuracy_Tg, 'prec_Tm' :prec_Tm, 'recall_Tm' : recall_Tm, 'accuracy_Tm' : accuracy_Tm }, columns=['prec_Tg','recall_Tg', 'accuracy_Tg', 'prec_Tm','recall_Tm', 'accuracy_Tm'])


In [31]:
df_performance['f1_score_Tg'] = 2*df_performance.prec_Tg*df_performance.recall_Tg/(df_performance.prec_Tg + df_performance.recall_Tg)
df_performance['f1_score_Tm'] = 2*df_performance.prec_Tm*df_performance.recall_Tm/(df_performance.prec_Tm + df_performance.recall_Tm)


In [32]:
df_performance

,prec_Tg,recall_Tg,accuracy_Tg,prec_Tm,recall_Tm,accuracy_Tm,f1_score_Tg,f1_score_Tm
0,0.976536,0.87751,0.878505,0.969512,0.797992,0.826794,0.924379,0.87543


In [19]:
best_values

{'num_layers': 2,
 'units_0': 512,
 'dropout_0': 0.5,
 'learning_rate': 0.001,
 'units_1': 256,
 'dropout_1': 0.1,
 'tuner/epochs': 300,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}